In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/DD2360/Assignment4/question3

/content/drive/MyDrive/DD2360/Assignment4/question3


In [ ]:
%%writefile heatEquation.cu
#include <cuda_runtime_api.h>
#include <math.h>
#include <stdlib.h>
#include <sys/time.h>
#include <cusparse_v2.h>
#include <cublas_v2.h>
#include <thrust/device_ptr.h>
#include <thrust/sequence.h>

#define gpuCheck(stmt)                                               \
  do {                                                               \
      cudaError_t err = stmt;                                        \
      if (err != cudaSuccess) {                                      \
          printf("ERROR. Failed to run stmt %s\n", #stmt);           \
          break;                                                     \
      }                                                              \
  } while (0)

// Macro to check the cuBLAS status
#define cublasCheck(stmt)                                            \
  do {                                                               \
      cublasStatus_t err = stmt;                                     \
      if (err != CUBLAS_STATUS_SUCCESS) {                            \
          printf("ERROR. Failed to run cuBLAS stmt %s\n", #stmt);    \
          break;                                                     \
      }                                                              \
  } while (0)

// Macro to check the cuSPARSE status
#define cusparseCheck(stmt)                                          \
  do {                                                               \
      cusparseStatus_t err = stmt;                                   \
      if (err != CUSPARSE_STATUS_SUCCESS) {                          \
          printf("ERROR. Failed to run cuSPARSE stmt %s\n", #stmt);  \
          break;                                                     \
      }                                                              \
  } while (0)


struct timeval t_start, t_end;
void cputimer_start(){
  gettimeofday(&t_start, 0);
}
void cputimer_stop(const char* info){
  gettimeofday(&t_end, 0);
  double time = (1000000.0*(t_end.tv_sec-t_start.tv_sec) + t_end.tv_usec-t_start.tv_usec);
  printf("Timing - %s. \t\tElasped %.0f microseconds \n", info, time);
}

double cputimer_stop_return(){
  gettimeofday(&t_end, 0);
  double time = (1000000.0*(t_end.tv_sec-t_start.tv_sec) + t_end.tv_usec-t_start.tv_usec);
  return time;
}

// Initialize the sparse matrix needed for the heat time step
void matrixInit(double* A, int* ArowPtr, int* AcolIndx, int dimX,
    double alpha) {
  // Stencil from the finete difference discretization of the equation
  double stencil[] = { 1, -2, 1 };
  // Variable holding the position to insert a new element
  size_t ptr = 0;
  // Insert a row of zeros at the beginning of the matrix
  ArowPtr[1] = ptr;
  // Fill the non zero entries of the matrix
  for (int i = 1; i < (dimX - 1); ++i) {
    // Insert the elements: A[i][i-1], A[i][i], A[i][i+1]
    for (int k = 0; k < 3; ++k) {
      // Set the value for A[i][i+k-1]
      A[ptr] = stencil[k];
      // Set the column index for A[i][i+k-1]
      AcolIndx[ptr++] = i + k - 1;
    }
    // Set the number of newly added elements
    ArowPtr[i + 1] = ptr;
  }
  // Insert a row of zeros at the end of the matrix
  ArowPtr[dimX] = ptr;
}

int main(int argc, char **argv) {
  int device = 0;            // Device to be used
  int dimX;                  // Dimension of the metal rod
  int nsteps;                // Number of time steps to perform
  double alpha = 0.4;        // Diffusion coefficient
  double* temp;              // Array to store the final time step
  double* A;                 // Sparse matrix A values in the CSR format
  int* ARowPtr;              // Sparse matrix A row pointers in the CSR format
  int* AColIndx;             // Sparse matrix A col values in the CSR format
  int nzv;                   // Number of non zero values in the sparse matrix
  double* tmp;               // Temporal array of dimX for computations
  size_t bufferSize = 0;     // Buffer size needed by some routines
  void* buffer = nullptr;    // Buffer used by some routines in the libraries
  int concurrentAccessQ;     // Check if concurrent access flag is set
  double zero = 0;           // Zero constant
  double one = 1;            // One constant
  double norm;               // Variable for norm values
  double error;              // Variable for storing the relative error
  double tempLeft = 200.;    // Left heat source applied to the rod
  double tempRight = 300.;   // Right heat source applied to the rod
  cublasHandle_t cublasHandle;      // cuBLAS handle
  cusparseHandle_t cusparseHandle;  // cuSPARSE handle
  //cusparseMatDescr_t Adescriptor;   // Mat descriptor needed by cuSPARSE
  cusparseSpMatDescr_t Adescriptor;
  // Read the arguments from the command line
  dimX = atoi(argv[1]);
  nsteps = atoi(argv[2]);

  // Print input arguments
  printf("The X dimension of the grid is %d \n", dimX);
  printf("The number of time steps to perform is %d \n", nsteps);

  // Get if the cudaDevAttrConcurrentManagedAccess flag is set
  gpuCheck(cudaDeviceGetAttribute(&concurrentAccessQ, cudaDevAttrConcurrentManagedAccess, device));

  // Calculate the number of non zero values in the sparse matrix. This number
  // is known from the structure of the sparse matrix
  nzv = 3 * dimX - 6;

  //@@ Insert the code to allocate the temp, tmp and the sparse matrix
  //@@ arrays using Unified Memory
  cputimer_start();
  gpuCheck(cudaMallocManaged(&temp, sizeof(double) * dimX));
  gpuCheck(cudaMallocManaged(&tmp, sizeof(double) * dimX));
  gpuCheck(cudaMallocManaged(&A, sizeof(double) * nzv));
  gpuCheck(cudaMallocManaged(&ARowPtr, sizeof(int*) * (dimX + 1)));
  gpuCheck(cudaMallocManaged(&AColIndx, sizeof(int*) * nzv));
  cputimer_stop("Allocating device memory");

  // Check if concurrentAccessQ is non zero in order to prefetch memory
  if (concurrentAccessQ) {
    cputimer_start();
    //@@ Insert code to prefetch in Unified Memory asynchronously to CPU
    gpuCheck(cudaMemPrefetchAsync(temp, dimX * sizeof(double), cudaCpuDeviceId, NULL));
    gpuCheck(cudaMemPrefetchAsync(tmp, dimX * sizeof(double), cudaCpuDeviceId, NULL));
    gpuCheck(cudaMemPrefetchAsync(A, nzv * sizeof(double), cudaCpuDeviceId, NULL));
    gpuCheck(cudaMemPrefetchAsync(ARowPtr,  sizeof(int*)*(dimX + 1) , cudaCpuDeviceId, NULL));
    gpuCheck(cudaMemPrefetchAsync(AColIndx,sizeof(int*) * nzv , cudaCpuDeviceId, NULL));
    cputimer_stop("Prefetching GPU memory to the host");
  }

  // Initialize the sparse matrix
  cputimer_start();
  matrixInit(A, ARowPtr, AColIndx, dimX, alpha);
  cputimer_stop("Initializing the sparse matrix on the host");

  //Initiliaze the boundary conditions for the heat equation
  cputimer_start();
  memset(temp, 0, sizeof(double) * dimX);
  temp[0] = tempLeft;
  temp[dimX - 1] = tempRight;
  cputimer_stop("Initializing memory on the host");

  if (concurrentAccessQ) {
    cputimer_start();
    //@@ Insert code to prefetch in Unified Memory asynchronously to the GPU
    gpuCheck(cudaMemPrefetchAsync(temp, sizeof(double) * dimX, device, NULL));
    gpuCheck(cudaMemPrefetchAsync(tmp, sizeof(double) * dimX, device, NULL));
    gpuCheck(cudaMemPrefetchAsync(A, sizeof(double) * nzv, device, NULL));
    gpuCheck(cudaMemPrefetchAsync(ARowPtr, sizeof(int*) * (dimX + 1), device, NULL));
    gpuCheck(cudaMemPrefetchAsync(AColIndx, sizeof(int*) * nzv, device, NULL));
    cputimer_stop("Prefetching GPU memory to the device");
  }

  //@@ Insert code to create the cuBLAS handle
  cublasCheck(cublasCreate(&cublasHandle));

  //@@ Insert code to create the cuSPARSE handle
  cusparseCheck(cusparseCreate(&cusparseHandle));

  //@@ Insert code to set the cuBLAS pointer mode to CUSPARSE_POINTER_MODE_HOST
  cublasCheck(cublasSetPointerMode(cublasHandle, CUBLAS_POINTER_MODE_HOST));

  //@@ Insert code to call cusparse api to create the mat descriptor used by cuSPARSE
  //Comes from the Nvdia sparse matrix example:https://github.com/NVIDIA/CUDALibrarySamples/blob/master/cuSPARSE/spmv_csr/spmv_csr_example.c
  // Create sparse matrix A in CSR format
  cusparseCheck(cusparseCreateCsr(&Adescriptor, dimX, dimX, nzv, ARowPtr, AColIndx, A,
                CUSPARSE_INDEX_32I, CUSPARSE_INDEX_32I, CUSPARSE_INDEX_BASE_ZERO, CUDA_R_32F));

  cusparseDnVecDescr_t vecX, vecY;
  // Create dense vector X
  cusparseCheck(cusparseCreateDnVec(&vecY, dimX, tmp, CUDA_R_32F));
  // Create dense vector y
  cusparseCheck(cusparseCreateDnVec(&vecX, dimX, temp, CUDA_R_32F));

  //@@ Insert code to call cusparse api to get the buffer size needed by the sparse matrix per
  //@@ vector (SMPV) CSR routine of cuSPARSE

  cusparseCheck(cusparseSpMV_bufferSize(cusparseHandle, CUSPARSE_OPERATION_NON_TRANSPOSE, &one, Adescriptor,
    vecX, &zero, vecY, CUDA_R_32F, CUSPARSE_SPMV_CSR_ALG1, &bufferSize));


  //@@ Insert code to allocate the buffer needed by cuSPARSE
  gpuCheck(cudaMalloc(&buffer, bufferSize));

  // Perform the time step iterations
  double timer = 0;
  for (int it = 0; it < nsteps; ++it) {
    //@@ Insert code to call cusparse api to compute the SMPV (sparse matrix multiplication) for
    //@@ the CSR matrix using cuSPARSE. This calculation corresponds to:
    //@@ tmp = 1 * A * temp + 0 * tmp
    cputimer_start();
    cusparseCheck(cusparseSpMV(cusparseHandle,CUSPARSE_OPERATION_NON_TRANSPOSE,&one,Adescriptor,vecX,&zero,vecY,CUDA_R_32F,
      CUSPARSE_SPMV_ALG_DEFAULT,buffer));
    timer += cputimer_stop_return();
    //@@ Insert code to call cublas api to compute the axpy routine using cuBLAS.
    //@@ This calculation corresponds to: temp = alpha * tmp + temp
    cublasCheck(cublasDaxpy(cublasHandle, dimX, &alpha, tmp,  1, temp, 1));

    //@@ Insert code to call cublas api to compute the norm of the vector using cuBLAS
    //@@ This calculation corresponds to: ||temp||
    cublasCheck(cublasDnrm2(cublasHandle, dimX, temp, 1, &norm));

    // If the norm of A*temp is smaller than 10^-4 exit the loop
    if (norm < 1e-4)
      break;
  }
  printf("timer:%.0f\n",timer);
  int flops = nsteps * nzv / (((int)timer)*1e-6);
  printf("The value of FLOPS in dimx:%d equals to %d\n gflops",dimX,flops);

  // Calculate the exact solution using thrust
  thrust::device_ptr<double> thrustPtr(tmp);
  thrust::sequence(thrustPtr, thrustPtr + dimX, tempLeft,
      (tempRight - tempLeft) / (dimX - 1));

  // Calculate the relative approximation error:
  one = -1;
  //@@ Insert the code to call cublas api to compute the difference between the exact solution
  //@@ and the approximation
  //@@ This calculation corresponds to: tmp = -temp + tmp
  cublasCheck(cublasDaxpy(cublasHandle, dimX, &one, temp, 1, tmp, 1));

  //@@ Insert the code to call cublas api to compute the norm of the absolute error
  //@@ This calculation corresponds to: || tmp ||
  cublasCheck(cublasDnrm2(cublasHandle, dimX, tmp, 1, &norm));

  error = norm;
  //@@ Insert the code to call cublas api to compute the norm of temp
  //@@ This calculation corresponds to: || temp ||
  cublasCheck(cublasDnrm2(cublasHandle, dimX, temp, 1, &norm));

  // Calculate the relative error
  error = error / norm;
  printf("The relative error of the approximation is %f\n", error);

  //@@ Insert the code to destroy the mat descriptor
  cusparseCheck(cusparseDestroySpMat(Adescriptor));

  //@@ Insert the code to destroy the cuSPARSE handle
  cusparseCheck(cusparseDestroy(cusparseHandle));

  //@@ Insert the code to destroy the cuBLAS handle
  cublasCheck(cublasDestroy(cublasHandle));


  //@@ Insert the code for deallocating memory
  cudaFree(temp);
  cudaFree(tmp);
  cudaFree(A);
  cudaFree(ARowPtr);
  cudaFree(AColIndx);
  cudaFree(buffer);

  return 0;
}




Overwriting heatEquation.cu


In [ ]:
!nvcc heatEquation.cu -lcublas -lcusparse
!ls

a.out  heatEquation.cu	sample_data


In [ ]:
!./a.out 65536 128

The X dimension of the grid is 65536 
The number of time steps to perform is 128 
Timing - Allocating device memory. 		Elasped 210282 microseconds 
Timing - Prefetching GPU memory to the host. 		Elasped 1596 microseconds 
Timing - Initializing the sparse matrix on the host. 		Elasped 1167 microseconds 
Timing - Initializing memory on the host. 		Elasped 78 microseconds 
Timing - Prefetching GPU memory to the device. 		Elasped 1656 microseconds 
timer:1172
The value of FLOPS in dimx:65536 equals to 21
The relative error of the approximation is 214.748561


In [ ]:
!./a.out 524288 128

The X dimension of the grid is 524288 
The number of time steps to perform is 128 
Timing - Allocating device memory. 		Elasped 180210 microseconds 
Timing - Prefetching GPU memory to the host. 		Elasped 8487 microseconds 
Timing - Initializing the sparse matrix on the host. 		Elasped 7289 microseconds 
Timing - Initializing memory on the host. 		Elasped 904 microseconds 
Timing - Prefetching GPU memory to the device. 		Elasped 544 microseconds 
timer:814
The value of FLOPS in dimx:524288 equals to 247
The relative error of the approximation is 607.406310


In [ ]:
!./a.out 4194304 128

The X dimension of the grid is 4194304 
The number of time steps to perform is 128 
Timing - Allocating device memory. 		Elasped 179684 microseconds 
Timing - Prefetching GPU memory to the host. 		Elasped 82041 microseconds 
Timing - Initializing the sparse matrix on the host. 		Elasped 54439 microseconds 
Timing - Initializing memory on the host. 		Elasped 7352 microseconds 
Timing - Prefetching GPU memory to the device. 		Elasped 2696 microseconds 
timer:1323
The value of FLOPS in dimx:4194304 equals to 1217
The relative error of the approximation is 1718.006484


In [ ]:
!./a.out 16777216 128

The X dimension of the grid is 16777216 
The number of time steps to perform is 128 
Timing - Allocating device memory. 		Elasped 184273 microseconds 
Timing - Prefetching GPU memory to the host. 		Elasped 259051 microseconds 
Timing - Initializing the sparse matrix on the host. 		Elasped 212982 microseconds 
Timing - Initializing memory on the host. 		Elasped 28932 microseconds 
Timing - Prefetching GPU memory to the device. 		Elasped 5018 microseconds 
timer:1090
The value of FLOPS in dimx:16777216 equals to 1970
The relative error of the approximation is 3436.013396


In [ ]:
!for i in {2..10..1}; do ./a.out $((2 ** i)) 10; done

The X dimension of the grid is 4 
The number of time steps to perform is 10 
Timing - Allocating device memory. 		Elasped 195041 microseconds 
Timing - Prefetching GPU memory to the host. 		Elasped 488 microseconds 
Timing - Initializing the sparse matrix on the host. 		Elasped 1 microseconds 
Timing - Initializing memory on the host. 		Elasped 1 microseconds 
Timing - Prefetching GPU memory to the device. 		Elasped 405 microseconds 
timer:322
The value of FLOPS in dimx:4 equals to 186335
 gflopsThe relative error of the approximation is 0.982757
The X dimension of the grid is 8 
The number of time steps to perform is 10 
Timing - Allocating device memory. 		Elasped 162895 microseconds 
Timing - Prefetching GPU memory to the host. 		Elasped 443 microseconds 
Timing - Initializing the sparse matrix on the host. 		Elasped 1 microseconds 
Timing - Initializing memory on the host. 		Elasped 0 microseconds 
Timing - Prefetching GPU memory to the device. 		Elasped 451 microseconds 
timer:357

In [ ]:
!for i in {100..10000..900}; do ./a.out 128 $i; done

The X dimension of the grid is 128 
The number of time steps to perform is 100 
Timing - Allocating device memory. 		Elasped 187048 microseconds 
Timing - Prefetching GPU memory to the host. 		Elasped 568 microseconds 
Timing - Initializing the sparse matrix on the host. 		Elasped 3 microseconds 
Timing - Initializing memory on the host. 		Elasped 1 microseconds 
Timing - Prefetching GPU memory to the device. 		Elasped 406 microseconds 
Timing - Computing the SMPV. 			Elasped 5755 microseconds 
The relative error of the approximation is 3.318021
The X dimension of the grid is 128 
The number of time steps to perform is 1000 
Timing - Allocating device memory. 		Elasped 171351 microseconds 
Timing - Prefetching GPU memory to the host. 		Elasped 459 microseconds 
Timing - Initializing the sparse matrix on the host. 		Elasped 3 microseconds 
Timing - Initializing memory on the host. 		Elasped 0 microseconds 
Timing - Prefetching GPU memory to the device. 		Elasped 374 microseconds 
Timing

with prefetching

In [ ]:
!nvprof ./a.out 128 1000


The X dimension of the grid is 128 
The number of time steps to perform is 1000 
==35770== NVPROF is profiling process 35770, command: ./a.out 128 1000
Timing - Allocating device memory. 		Elasped 244708 microseconds 
Timing - Prefetching GPU memory to the host. 		Elasped 609 microseconds 
Timing - Initializing the sparse matrix on the host. 		Elasped 5 microseconds 
Timing - Initializing memory on the host. 		Elasped 0 microseconds 
Timing - Prefetching GPU memory to the device. 		Elasped 437 microseconds 
Timing - Computing the SMPV. 			Elasped 26123 microseconds 
The relative error of the approximation is 1.488118
==35770== Profiling application: ./a.out 128 1000
==35770== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   80.31%  92.986ms      2004  46.400us  32.959us  54.655us  void nrm2_kernel<double, double, double, int=0, int=0, int=128>(cublasNrm2Params<int, double, double>)
                   10.24%  11.851ms

without prefetching

In [ ]:
!nvprof ./a.out 128 1000

The X dimension of the grid is 128 
The number of time steps to perform is 1000 
==36128== NVPROF is profiling process 36128, command: ./a.out 128 1000
Timing - Allocating device memory. 		Elasped 222999 microseconds 
Timing - Initializing the sparse matrix on the host. 		Elasped 583 microseconds 
Timing - Initializing memory on the host. 		Elasped 1 microseconds 
Timing - Computing the SMPV. 			Elasped 22360 microseconds 
The relative error of the approximation is 1.488118
==36128== Profiling application: ./a.out 128 1000
==36128== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   80.29%  97.670ms      2004  48.737us  44.383us  54.686us  void nrm2_kernel<double, double, double, int=0, int=0, int=128>(cublasNrm2Params<int, double, double>)
                   10.14%  12.338ms      1000  12.337us  12.192us  13.408us  void cusparse::csrmv_v3_kernel<bool=0, int, int, double, double, double, double, void>(cusparse::KernelC